In [1]:
from Gaussian import Gaussian
from EM import EM
from NN import NN
import numpy as np

def complex_train(D, K):
    method_number = 5

    # parameters for neural networks
    neuron_1     = {0: K}
    neuron_2_10  = {0: 10, 1: K}
    neuron_2_100 = {0: 100, 1: K}
    act_func_1 = {0: NN.softmax}
    act_func_2 = {0: NN.relu, 1: NN.softmax}

    # store the result
    accuracy = np.zeros(method_number)
    time     = np.zeros(method_number)

    # generate sample
    G = Gaussian(D=D, K=K, background=False, index_para=[6000, 9000])

    # train and test each method, [ EM, Q, C100, C10, C ]
    method_set = [EM(K),
                  NN(D, neuron_1, act_func_1, NN_type="QNN"),
                  NN(D, neuron_2_100, act_func_2, NN_type="CNN"),
                  NN(D, neuron_2_10, act_func_2, NN_type="CNN"),
                  NN(D, neuron_1, act_func_1, NN_type="CNN")]
    for j in range(method_number):
        method = method_set[j]
        if j == 0:
            method.train(G.train_point)
            method.order_correction(G.valid_point, G.valid_label)
        else:
            method.train(G.train_point, G.train_label,
                         G.valid_point, G.valid_label, step_size=500)

        accuracy[j] = method.test(G.test_point, G.test_label)[0] * 100
        time[j]     = method.train_time

    return accuracy, time   # EM, Q, C100, C10, C

In [2]:
# start training and store all the result

accuracy_25, accuracy_35, accuracy_28, accuracy_38 = [], [], [], []
time_25, time_35, time_28, time_38 = [], [], [], []

sample_number = 50
for _ in range(sample_number):
    # D = 2, K = 5
    accuracy, time = complex_train(D=2, K=5)
    accuracy_25.append(accuracy)
    time_25.append(time)

    # D = 3, K = 5
    accuracy, time = complex_train(D=3, K=5)
    accuracy_35.append(accuracy)
    time_35.append(time)

    # D = 2, K = 8
    accuracy, time = complex_train(D=2, K=8)
    accuracy_28.append(accuracy)
    time_28.append(time)

    # D = 3, K = 8
    accuracy, time = complex_train(D=3, K=8)
    accuracy_38.append(accuracy)
    time_38.append(time)

In [3]:
# print result

string = ["EM        ",  "Q(2-3)    ", "C(2-100-3)", "C(2-10-3) ", "C(2-3)    "]

print("\nD=2, K=5\t\tAccuracy\t\t\t\tTime")
for i in range(4, -1, -1):
    meanA, varianceA = np.mean(accuracy_25[:, i]), np.std(accuracy_25[:, i])
    meanT, varianceT = np.mean(time_25[:, i]), np.std(time_25[:, i])
    print("%s:\t%2.2f\t+-\t%2.2f\t\t%2.2f\t+-\t%2.2f" %
          (string[i], meanA, varianceA, meanT, varianceT))

print("\nD=3, K=5\t\tAccuracy\t\t\t\tTime")
for i in range(4, -1, -1):
    meanA, varianceA = np.mean(accuracy_35[:, i]), np.std(accuracy_35[:, i])
    meanT, varianceT = np.mean(time_35[:, i]), np.std(time_35[:, i])
    print("%s:\t%2.2f\t+-\t%2.2f\t\t%2.2f\t+-\t%2.2f" %
          (string[i], meanA, varianceA, meanT, varianceT))

print("\nD=2, K=8\t\tAccuracy\t\t\t\tTime")
for i in range(4, -1, -1):
    meanA, varianceA = np.mean(accuracy_28[:, i]), np.std(accuracy_28[:, i])
    meanT, varianceT = np.mean(time_28[:, i]), np.std(time_28[:, i])
    print("%s:\t%2.2f\t+-\t%2.2f\t\t%2.2f\t+-\t%2.2f" %
          (string[i], meanA, varianceA, meanT, varianceT))

print("\nD=3, K=8\t\tAccuracy\t\t\t\tTime")
for i in range(4, -1, -1):
    meanA, varianceA = np.mean(accuracy_38[:, i]), np.std(accuracy_38[:, i])
    meanT, varianceT = np.mean(time_38[:, i]), np.std(time_38[:, i])
    print("%s:\t%2.2f\t+-\t%2.2f\t\t%2.2f\t+-\t%2.2f" %
          (string[i], meanA, varianceA, meanT, varianceT))


D=2, K=5		Accuracy				Time
C(2-3)    :	92.36	+-	7.89		11.54	+-	5.04
C(2-10-3) :	95.36	+-	5.66		23.28	+-	14.13
C(2-100-3):	95.47	+-	5.68		62.71	+-	27.90
Q(2-3)    :	95.53	+-	5.66		15.83	+-	6.70
EM        :	95.60	+-	5.65		1.82	+-	1.06

D=3, K=5		Accuracy				Time
C(2-3)    :	84.72	+-	5.53		13.74	+-	5.75
C(2-10-3) :	92.15	+-	3.75		24.79	+-	8.61
C(2-100-3):	92.54	+-	3.70		71.80	+-	29.22
Q(2-3)    :	92.74	+-	3.67		17.56	+-	7.36
EM        :	92.90	+-	3.64		0.78	+-	0.93

D=2, K=8		Accuracy				Time
C(2-3)    :	83.84	+-	10.32		22.79	+-	8.01
C(2-10-3) :	87.90	+-	8.04		47.62	+-	17.05
C(2-100-3):	88.06	+-	8.01		122.59	+-	39.24
Q(2-3)    :	88.13	+-	7.98		33.26	+-	10.97
EM        :	88.19	+-	7.97		6.81	+-	1.59

D=3, K=8		Accuracy				Time
C(2-3)    :	76.47	+-	6.31		28.83	+-	10.60
C(2-10-3) :	82.45	+-	5.36		62.62	+-	18.46
C(2-100-3):	82.68	+-	5.34		151.84	+-	41.37
Q(2-3)    :	82.79	+-	5.31		46.24	+-	13.33
EM        :	82.86	+-	5.32		2.77	+-	2.11
